## ResNet 做图像分类

### 需要载入的库

我们知道的一个共识是，越深的网络越能够表示复杂的函数，也就越能训练更有表现力的网络。但是一般这样的网络很难训练：
- 因为网络太深容易导致梯度消失或者爆炸,尤其是梯度消失，
- 同时训练很慢。
为了解决这个网络，我们找到了一种网络如下：
Residual Network 可以实现深层网络，而且同样可以获得很好的训练结果。
这里我们利用ResNet 来实现人脸微笑检测模型。
需要导入如下库：

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


### 创建一个ResNet

首先我们来看看ResNet张什么样子，如下：

![](imgs/2.jpg)

就是在普通的网络基础上添加了一个“shortcut”或“skip connection”层，这样可以将前面层的数据直接传给后面的层。
这样子就会很容易获得一个恒等函数 X(L) = X(L+2),具体的流程如下：
![](imgs/3.jpg)

#### 创建恒等模块

下面我们来看看一个标准的ResNet 块是什么样子：
![](imgs/4.jpg)

从图中我们看到其包括两个部分：
- main path: 就是由：CONV->BN->Relu->Conv->BN组成
- shortcut path: 直接将X(L-2)传递到X的rule之前

下面我们这里将会使用3层的块，具体如下图：
![](imgs/5.jpg)

从图中我们看到其包括两个部分：
- main path: 就是由：CONV->BN->Relu->Conv->BN->Relu->Conv->BN->Add
- shortcut path: 直接将X(L-3)传递到X的rule之前

下面来看看其具体的组成：

main path的第一层
- CONV2D F1滤波器，
    - shape(1,1),
    - stride(1,1),
    - 名字为"conv_name_base + '2a'"
    - 使用随机初始化
- Batch Norm
    - 名字为"bn_name_base + '2a'"
- Relu

main path的第二层
- CONV2D F2滤波器，
    - shape(f,f),
    - stride(1,1),
    - 名字为"conv_name_base + '2b'"
    - 使用随机初始化
- Batch Norm
    - 名字为"bn_name_base + '2b'"
- Relu

main path的第三层
- CONV2D F3滤波器，
    - shape(1,1),
    - stride(1,1),
    - 名字为"conv_name_base + '2c'"
    - 使用随机初始化
- Batch Norm
    - 名字为"bn_name_base + '2c'"

最后一步是：
- 利用shortcut将input 与后面的结果相加
- Relu


具体实现的过程如下：

In [2]:
def identity_block(X, f, filters, stage, block):
    """
    参数：
        X: input
        f:表示第二个fliter的size
        filters: 是一个数组，表示各个fliter的数量
        stage: 表示目前在网络的那个阶段
        block: 表示所在阶段的第几个块
    """
    
    # 定义目前网络所在的层与那个block
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 获得滤波器
    F1, F2, F3 = filters
    
    # 保存input value
    X_shortcut = X
    
    # mainpath:第一个组件
    X = Conv2D(filters=F1,
               kernel_size=(1,1),
               strides=(1,1),
               padding='valid',
               name = conv_name_base + '2a',
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis = 3,
                           name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)
    
    # mainpath:第二个组件
    X = Conv2D(filters=F2,
               kernel_size=(f,f),
               strides=(1,1),
               padding='same',
               name = conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis = 3,
                           name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)
    
    # mainpath:第三个组件
    X = Conv2D(filters=F3,
               kernel_size=(1,1),
               strides=(1,1),
               padding='valid',
               name = conv_name_base + '2c',
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis = 3,
                           name=bn_name_base+'2c')(X)
    
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X
    
    

下面测试函数，有没有发现一个有趣的现象，虽然这里使用keras了编写的函数，但是我们测试的时候用的是tensorflow来运行，因为keras这里的后台用的就是tensorflow的API,这里的keras就是一个高级封装的tf.所以使用tf可以执行该函数。  
函数解析：
K.learning_phase()
- 决定当前模型执行于训练模式：0
- 还是测试模式：1

In [6]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X,K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [0.94822997 0.         1.1610144  2.747859   0.         1.36677   ]


#### 创建卷积块

上面创建的是标准的恒等块，但是有没有发现一个问题，就是如果当我们在恒等模块中最后相加的步骤中，发现X(L-2)与X(L)维度或则shape不同的时候，就无法直接相加了。这里我们就需要对X(L-2)进行处理，保证与X(L)的维度一致。所以这里我们提出在shortcut层也叠加一个卷积模块，来实现各个维度的一致性。  

这里我们可以通过加入一个1x1的Conv2D实现。

![](imgs/6.jpg)

具体的过程如下：

**main path的第一层**
- CONV2D F1滤波器，
    - shape(1,1),
    - stride(1,1),
    - 名字为"conv_name_base + '2a'"
    - 使用随机初始化
- Batch Norm
    - 名字为"bn_name_base + '2a'"
- Relu

**main path的第二层**
- CONV2D F2滤波器，
    - shape(f,f),
    - stride(1,1),
    - 名字为"conv_name_base + '2b'"
    - 使用随机初始化
- Batch Norm
    - 名字为"bn_name_base + '2b'"
- Relu

**main path的第三层**
- CONV2D F3滤波器，
    - shape(1,1),
    - stride(1,1),
    - 名字为"conv_name_base + '2c'"
    - 使用随机初始化
- Batch Norm
    - 名字为"bn_name_base + '2c'"
    
 **shortcut层**   
- CONV2D F3滤波器，
    - shape(1,1),
    - stride(s,s),
    - 名字为"conv_name_base + '1'"
    - 使用随机初始化
- Batch Norm
    - 名字为"bn_name_base + '1'"

最后一步是：
- 利用shortcut将input 与后面的结果相加
- Relu

coding如下:


In [3]:
def convolutional_block(X, f, filters, stage, block,s=2):
    """
    参数：
        X: input
        f:表示第二个fliter的size
        filters: 是一个数组，表示各个fliter的数量
        stage: 表示目前在网络的那个阶段
        block: 表示所在阶段的第几个块
        s:表示strides
    """
    
    # 定义目前网络所在的层与那个block
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 获得滤波器
    F1, F2, F3 = filters
    
    # 保存input value
    X_shortcut = X
    
    # mainpath:第一个组件
    X = Conv2D(filters=F1,
               kernel_size=(1,1),
               strides=(s,s),
               padding='valid',
               name = conv_name_base + '2a',
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis = 3,
                           name=bn_name_base+'2a')(X)
    X = Activation('relu')(X)
    
    # mainpath:第二个组件
    X = Conv2D(filters=F2,
               kernel_size=(f,f),
               strides=(1,1),
               padding='same',
               name = conv_name_base + '2b',
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis = 3,
                           name=bn_name_base+'2b')(X)
    X = Activation('relu')(X)
    
    # mainpath:第三个组件
    X = Conv2D(filters=F3,
               kernel_size=(1,1),
               strides=(1,1),
               padding='valid',
               name = conv_name_base + '2c',
               kernel_initializer=glorot_uniform(seed=0)
              )(X)
    X = BatchNormalization(axis = 3,
                           name=bn_name_base+'2c')(X)
    
    # shortcut 层
    X_shortcut = Conv2D(filters=F3,
                           kernel_size=(1,1),
                           strides=(s,s),
                           padding='valid',
                           name = conv_name_base + '1',
                           kernel_initializer=glorot_uniform(seed=0)
                          )(X_shortcut) 
    X_shortcut = BatchNormalization(axis = 3,
                           name=bn_name_base+'1')(X_shortcut)
    

    # 最后一层叠加
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X
    
    

In [11]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))
    print(A.shape)

out = [0.09018463 1.2348979  0.46822023 0.03671762 0.         0.65516603]
(3, 2, 2, 6)


### 搭建ResNet50模型

上面我们基本都搭好两个基础的函数了，现在我们要开始搭积木，我们将搭一个50层网络的模型ResNet模型，这里我们将上面的Identity block简称为ID block. 模型的结构如下图所示：

![](imgs/7.jpg)

上图中包含了几个stage,具体的过程详细如下：

- **Zero-padding**
- Zero-padding pads (3,3)

- **stage 1**
    - Conv2D f=64,shape=(7,7),stride=(2，2)，name="conv1"
    - BN
    - MaxPooling window=(3,3) stride = (2,2)

- **stage 2**
    - Conv2D f=256,shape=(3,3),stride=(1，1)，block = "a"
    - 2 x ID blocks shape(3,3),stride=(2，2), block = "b","c"
    
- **stage 3**
    - Conv2D f=512,shape=(3,3),stride=(2，2)，block = "a"
    - 3 x ID blocks shape(3,3),stride=(1，1), block = "b","c","d"
    
- **stage 4**
    - Conv2D f=1024,shape=(3,3),stride=(2，2)，block = "a"
    - 5 x ID blocks shape(3,3),stride=(1，1), block = "b","c","d",“e”,"f"
    
### 搭建ResNet50模型

上面我们基本都搭好两个基础的函数了，现在我们要开始搭积木，我们将搭一个50层网络的模型ResNet模型，这里我们将上面的Identity block简称为ID block. 模型的结构如下图所示：

![](imgs/7.jpg)

上图中包含了几个stage,具体的过程详细如下：

- **Zero-padding**
- Zero-padding pads (3,3)

- **stage 1**
    - Conv2D f=64,shape=(7,7),stride=(2，2)，name="conv1"
    - BN
    - MaxPooling window=(3,3) stride = (2,2)

- **stage 2**
    - Conv2D f=256,shape=(3,3),stride=(1，1)，block = "a"
    - 2 x ID blocks shape(3,3),stride=(2，2), block = "b","c"
    
- **stage 3**
    - Conv2D f=512,shape=(3,3),stride=(2，2)，block = "a"
    - 3 x ID blocks shape(3,3),stride=(1，1), block = "b","c","d"
    
- **stage 4**
    - Conv2D f=1024,shape=(3,3),stride=(2，2)，block = "a"
    - 5 x ID blocks shape(3,3),stride=(1，1), block = "b","c","d",“e”,"f"
    
- **stage 5**
    - Conv2D f=2048,shape=(3,3),stride=(2，2)，block = "a"
    - 5 x ID blocks shape(3,3),stride=(1，1), block = "b","c"
    
- **final**
    - average pooling window = (2,2) stride = (2,2)
    - flatten
    - FC 

具体实现的coding如下：

In [6]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    
    X_input = Input(input_shape)
    
    # 零填充
    X = ZeroPadding2D((3,3))(X_input)
    
    # 阶段1：
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3,name='bn_conv1')(X)
    X = MaxPooling2D((3,3),strides=(2,2))(X)
    
    # 阶段2：
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    
    # 阶段3：
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    
    # 阶段4：
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    
    # 阶段5：
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    
    X = AveragePooling2D((2,2),name="avg_pool")(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [7]:
# 创建模型
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### 利用ResNet50模型来实现手势识别

数据的导入：

In [10]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# 归一化
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# reshape
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


训练模型，这里要注意，请大家尽可能使用GPU来训练，使用cpu的熟读有点慢。

In [ ]:
# 训练该模型
model.fit(X_train, Y_train, epochs = 20, batch_size = 32)
model.save("ResNet50-1.h5")

In [ ]:
# 测试模型
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))